In [129]:
import pandas as pd
import numpy as np

import ast
import nltk
import ssl
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import jaccard_score



from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, expr, split
from pyspark.sql.types import FloatType, StringType
from pyspark.sql.functions import col

In [130]:
# Download necessary nltk packages for language processing
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andresrivera/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/andresrivera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/andresrivera/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [131]:
movies_df = pd.read_csv("final_movie.csv")

sample = movies_df.sample(n=2000, random_state=42)

In [63]:
sample

,id,keywords,cast,genres,imdb_id,original_language,revenue,release_date,spoken_languages,title,vote_average,vote_count,overview,poster_path,popularity,Director
15568,46563,revenge gambler creole,GaryCooper IngridBergman FloraRobson JerryAust...,"['Drama', 'Romance']",tt0038053,en,0.0,1945,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Saratoga Trunk,4.4,6.0,An opportunistic Texas gambler and the exiled ...,/x1p47FRX7dbPoaYMK6ANw8WlAC5.jpg,0.379884,['Sam Wood']
30182,67378,NaN,UgoTognazzi MarinaVlady WalterGiller LindaSini,"['Drama', 'Comedy']",tt0057532,it,0.0,1963,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",The Conjugal Bed,4.9,5.0,"A 40 years old man, Alfonso, finally succeeds ...",/fDVgCYMAyGRLagXQsQL2VRxfp6I.jpg,0.190725,['Marco Ferreri']
30271,73353,NaN,FlexAlexander AmySloan KristaRae EugeneClark,"['Drama', 'Documentary']",tt0411377,en,0.0,2004,"[{'iso_639_1': 'en', 'name': 'English'}]",Man in the Mirror: The Michael Jackson Story,5.0,3.0,Chronicles the rise and fall of pop king Micha...,/lVLcBwLWZUMOkpIalo5l5z8JT82.jpg,0.983500,[]
33535,184440,NaN,CoreySevier JordanLadd KimPoirier JamesGleason...,"['Drama', 'Romance', 'Science Fiction']",tt1894616,en,0.0,2013,"[{'iso_639_1': 'en', 'name': 'English'}]",Awaken,7.1,7.0,Alex's dreary life is reignited when he meets ...,/veCxc1ktBj5p42YGLzDlE9C2KYr.jpg,0.837087,['Daric Loo']
11837,13074,sport,SamuelL.Jackson JoshHartnett AlanAlda TeriHatc...,['Drama'],tt0416185,en,0.0,2007,"[{'iso_639_1': 'en', 'name': 'English'}]",Resurrecting the Champ,5.9,62.0,Up-and-coming sports reporter rescues a homele...,/18epMloEUD83jCNQDHr8dRTaM8U.jpg,7.451281,['Rod Lurie']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13721,18696,invasion ancientgreece greeksoldier,SteveReeves MylèneDemongeot SergioFantoni Dani...,"['Action', 'Drama']",tt0052604,it,0.0,1959,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",The Giant of Marathon,4.8,4.0,A Greek soldier leads the fight against an inv...,/hsvwUCKywHYp8qBEEOqYB722L3i.jpg,1.668343,"['Jacques Tourneur', 'Mario Bava']"
17212,54384,NaN,SóniaBandeira FábioOliveira ManuelSoares Joaqu...,"['Music', 'Romance']",tt1081929,pt,0.0,2008,"[{'iso_639_1': 'pt', 'name': 'Português'}]",Our Beloved Month of August,7.0,8.0,"In the heart of Portugal, amid the mountains, ...",/babgHk4BHbT005fnVDhZCNhl32j.jpg,0.866608,['Miguel Gomes']
13361,18094,canadian heavymetal friendship rockband hardro...,Steve'Lips'Kudlow RobbReiner KevinGoocher Glen...,"['Documentary', 'Music']",tt1157605,en,0.0,2008,"[{'iso_639_1': 'en', 'name': 'English'}]",Anvil! The Story of Anvil,7.7,85.0,"At 14, best friends Robb Reiner and Lips made ...",/rDrMuFaWBKwjwEnw4C06Sm97huY.jpg,3.414041,['Sacha Gervasi']
14010,91262,NaN,GeorgesHubert NicoleLadmiral AlfredMacquart Ma...,"['Documentary', 'Horror']",tt0041842,fr,0.0,1949,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Blood of the Beasts,8.0,18.0,"An early example of ultra-realism, this movie ...",/v3gTStmXJAZCSkEZmV6FURXQBve.jpg,1.235239,['Georges Franju']


In [132]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 26689 to 22796
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2000 non-null   int64  
 1   keywords           1575 non-null   object 
 2   cast               1969 non-null   object 
 3   genres             2000 non-null   object 
 4   imdb_id            2000 non-null   object 
 5   original_language  2000 non-null   object 
 6   revenue            2000 non-null   float64
 7   release_date       2000 non-null   int64  
 8   spoken_languages   2000 non-null   object 
 9   title              2000 non-null   object 
 10  vote_average       2000 non-null   float64
 11  vote_count         2000 non-null   float64
 12  overview           1991 non-null   object 
 13  poster_path        2000 non-null   object 
 14  popularity         2000 non-null   float64
 15  Director           2000 non-null   object 
dtypes: float64(4), int6

In [133]:


sample['cast'].fillna("", inplace=True) # replace missing values with empty string
sample['keywords'].fillna("", inplace=True) # replace missing values with empty string


sample['cast'].fillna("", inplace=True)



In [134]:
sample.reset_index(inplace=True)
sample.drop(['index'], axis=1, inplace=True)

In [135]:
# Processing text data
def preprocess_overview(text):

    if not isinstance(text, str):
        return []
    
    # tokenize the text
    tokens = word_tokenize(text)

     # Convert to lowercase and remove non-alphanumeric characters
    words = [word.lower() for word in tokens if word.isalnum()]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]
    
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    
    return lemmatized_words

In [136]:
def extract_keywords(row):
    keywords = str(row['keywords'])
    
    
    directors = ' '.join(ast.literal_eval(row['Director']))
    
    overview = row['overview']
    overview_keywords = ' '.join(preprocess_overview(overview))

    genres = ' '.join(ast.literal_eval(row['genres'])).lower()

    year = str(row['release_date'])

    cast = row['cast']


    
    combined_keywords = ' '.join([keywords, year, overview_keywords, directors, cast])
    
    return combined_keywords


In [ ]:

df_movies = pd.DataFrame()


sample['combined_keywords'] = sample.apply(extract_keywords, axis=1)
sample['title'] = sample['title']
sample['genres'] = sample['genres']

In [138]:
sample['genres_set'] = sample.genres.apply(lambda genres: set(ast.literal_eval(genres)))

In [139]:
sample["genres_set_str"] = sample["genres_set"].apply(lambda x: ' '.join(x))
sample.drop(['genres_set'], axis=1, inplace=True)

In [140]:
sample["movie_index"] = sample.index


In [141]:
spark = SparkSession.builder.master("local[*]").appName("MovieRecommendation").getOrCreate()
spark_movies = spark.createDataFrame(sample)



In [22]:
spark.stop()

In [144]:
# Define the Jaccard similarity function as a UDF
@udf(returnType=FloatType())
def jaccard_similarity_udf(set1_str, set2_str):
    set1 = set(set1_str.split())
    set2 = set(set2_str.split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0




# Calculate the Jaccard similarity matrix using a Cartesian join
spark_movies_with_similarity = (
    spark_movies.alias("movies1")
    .crossJoin(spark_movies.alias("movies2"))
    .select(
        col("movies1.movie_index").alias("movie1_index"),
        col("movies2.movie_index").alias("movie2_index"),
        jaccard_similarity_udf("movies1.genres_set_str", "movies2.genres_set_str").alias("jaccard_similarity"),
    )
)


In [145]:
spark_movies_with_similarity.show()

+------------+------------+------------------+
|movie1_index|movie2_index|jaccard_similarity|
+------------+------------+------------------+
|           0|           0|               1.0|
|           0|           1|        0.16666667|
|           0|           2|        0.16666667|
|           0|           3|               0.0|
|           0|           4|               0.0|
|           0|           5|               0.0|
|           0|           6|        0.16666667|
|           0|           7|               0.2|
|           0|           8|               0.0|
|           0|           9|               0.0|
|           0|          10|               0.0|
|           0|          11|              0.25|
|           0|          12|               0.0|
|           0|          13|             0.125|
|           0|          14|               0.2|
|           0|          15|               0.0|
|           0|          16|               0.2|
|           0|          17|               0.0|
|           0

In [146]:

pd_movies_with_similarity = spark_movies_with_similarity.toPandas()


jaccard_similarity_matrix = pd_movies_with_similarity.pivot_table(
    index="movie1_index",
    columns="movie2_index",
    values="jaccard_similarity"
).values


In [79]:
jaccard_similarity_matrix.shape

(1000, 1000)

In [147]:
stop = list(stopwords.words('english'))

tfidf_vectorizer = TfidfVectorizer(analyzer = 'word', stop_words=list(set(stop)))
tfidf_matrix = tfidf_vectorizer.fit_transform(sample['combined_keywords'])

cosine_sim = cosine_similarity(tfidf_matrix)

In [38]:
cosine_sim.shape

(1000, 1000)

In [148]:
# Define weighting for cosine and jaccard similarity scores

jaccard_weight = 0.25
cosine_weight = 1 - jaccard_weight

combined_matrix = (jaccard_weight * jaccard_similarity_matrix) + (cosine_weight * cosine_sim)

In [149]:
combined_matrix

array([[1.        , 0.04536956, 0.04616902, ..., 0.04479357, 0.        ,
        0.        ],
       [0.04536956, 1.        , 0.05656896, ..., 0.125     , 0.07709899,
        0.03249385],
       [0.04616902, 0.05656896, 1.        , ..., 0.125     , 0.0625    ,
        0.00926787],
       ...,
       [0.04479357, 0.125     , 0.125     , ..., 1.        , 0.0625    ,
        0.        ],
       [0.        , 0.07709899, 0.0625    , ..., 0.0625    , 1.        ,
        0.05773397],
       [0.        , 0.03249385, 0.00926787, ..., 0.        , 0.05773397,
        1.        ]])

In [151]:
def recommend_movies(title, num_recommendations=10):
    # Get the index of the movie with the given title
    movie_index = sample[sample['title'] == title].index[0]

    # Get the list of cosine similarity scores for the given movie
    similarity_scores = list(enumerate(combined_matrix[movie_index]))

    # Sort the scores in descending order
    similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the most similar movies (excluding the input movie itself)
    similar_movie_indices = [score[0] for score in similarity_scores_sorted[1:num_recommendations + 1]]

    # Return the titles of the most similar movies
    return sample['title'].iloc[similar_movie_indices]

In [47]:
movie_index = sample[sample['title'] == "Hannibal Rising"].index[0]

movie_index

995

In [ ]:
sample[sample['release_date'] == '2013']

In [152]:
recommended_movies = recommend_movies("Mr. Bean's Holiday", 10)
print(recommended_movies)

1975                      A Christmas Story
1535                   Jingle All the Way 2
219                             The Funeral
1505    Diary of a Wimpy Kid: The Long Haul
216                               Nativity!
1731                        Life With Mikey
827                     Alone For Christmas
1090    Diary of a Wimpy Kid: Rodrick Rules
179                        Stanley Ka Dabba
20                           Spooky Buddies
Name: title, dtype: object


In [154]:
mat = np.matrix(combined_matrix)

mat_df = pd.DataFrame(data=mat.astype(float))

In [156]:
mat_df

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,1.000000,0.045370,0.046169,0.00000,0.004542,0.000000,0.041667,0.050000,0.006731,0.007784,...,0.055803,0.050000,0.035714,0.045236,0.074890,0.000000,0.006462,0.044794,0.000000,0.000000
1,0.045370,1.000000,0.056569,0.00000,0.000000,0.000000,0.054084,0.062500,0.009307,0.050000,...,0.175092,0.076976,0.041667,0.125000,0.083333,0.007322,0.000000,0.125000,0.077099,0.032494
2,0.046169,0.056569,1.000000,0.00000,0.000000,0.007144,0.003006,0.000000,0.000000,0.129733,...,0.062500,0.005018,0.100000,0.010722,0.000000,0.008301,0.000000,0.125000,0.062500,0.009268
3,0.000000,0.000000,0.000000,1.00000,0.250000,0.000000,0.086890,0.004279,0.000000,0.083333,...,0.029948,0.000000,0.062500,0.000000,0.000000,0.125000,0.250000,0.000000,0.125000,0.088540
4,0.004542,0.000000,0.000000,0.25000,1.000000,0.000000,0.083333,0.000000,0.000000,0.083333,...,0.000000,0.000000,0.062500,0.000000,0.000000,0.132516,0.250000,0.000000,0.130717,0.062500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.000000,0.007322,0.008301,0.12500,0.132516,0.000000,0.071638,0.000000,0.000000,0.062500,...,0.010550,0.009993,0.058309,0.000000,0.000000,1.000000,0.147623,0.000000,0.088772,0.132897
1996,0.006462,0.000000,0.000000,0.25000,0.250000,0.000000,0.088472,0.000000,0.000000,0.083333,...,0.014703,0.000000,0.067903,0.000000,0.000000,0.147623,1.000000,0.000000,0.125000,0.062500
1997,0.044794,0.125000,0.125000,0.00000,0.000000,0.000000,0.050000,0.062500,0.005808,0.131098,...,0.166667,0.065475,0.041667,0.085107,0.087883,0.000000,0.000000,1.000000,0.062500,0.000000
1998,0.000000,0.077099,0.062500,0.12500,0.130717,0.000000,0.064963,0.010671,0.000000,0.166667,...,0.089592,0.000000,0.125000,0.000000,0.005415,0.088772,0.125000,0.062500,1.000000,0.057734


In [166]:
mat_df.to_csv('graph_db/data/scores_matrix.csv', sep=' ', header=True, index=False)
sample.to_csv("graph_db/data/sample.csv")




# Also add data to the movie_app directory
mat_df.to_csv('movie_app/data/scores_matrix.csv', sep=' ', header=True, index=False)
sample.to_csv("movie_app/data/sample.csv")
